# 2. Machine Learning

1. Построить модель (при помощи Python) прогнозирование дефолта клиента на аппликационных и поведенческих данных из сайта с помощью логистической регрессии, спрогнозировать вероятность не выполнения обязательств перед компанией. Финальную модель логистической регрессии необходимо отобразить в Excel (параметры, группы и баллы).

In [1]:
import gc

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from pprint import pprint

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import auc, roc_curve, accuracy_score, f1_score


# Reading the dataset

In [2]:
# %%time
# data = pd.read_excel('data.xlsx')

#takes 2 mins to load

In [3]:
%%time 
data = pd.read_csv('data.csv')

#takes 2 seconds

CPU times: user 1.93 s, sys: 172 ms, total: 2.11 s
Wall time: 2.24 s


In [4]:
print('data shape:', data.shape)

data shape: (35311, 285)


In [5]:
data.head()

,target_flag,crdeal_deallife__loans_cnt__dlrolesub_zaemshik__curr_uah,crdeal_deallife__loans_cnt__dlrolesub_zaemshik__curr_not_uah,crdeal_deallife__loans_cnt__dldonor_bnk,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_7_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_30_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_45_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_60_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_90_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_120_days,...,credres__credit_req_cnt__org_bnk__last_3_5_days,credres__credit_req_cnt__org_bnk__last_7_days,credres__credit_req_cnt__org_bnk__last_10_days,credres__credit_req_cnt__org_bnk__last_13_days,credres__credit_req_cnt__org_bnk__last_20_days,credres__credit_req_cnt__org_bnk__last_30_days,credres__credit_req_cnt__org_bnk__last_60_days,credres__credit_req_cnt__org_bnk__more_than_90_days,credres__credit_req_cnt__org_bnk__last_360_days,credres__credit_req_cnt__org_bnk__more_than_360_days
0,good,40.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,5.0,4.0
1,good,72.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,14.0,6.0,11.0
2,good,77.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,5.0,9.0
3,good,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0
4,good,8.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


# Handling missing data

In [6]:
missing = data.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)

In [7]:
missing

crdeal_deallife__loans_cnt__dlrolesub_zaemshik__curr_uah                     94
crdeal_deallife__loans_cnt__state_discounted__last_active_lteq_90_days       94
crdeal_deallife__loans_cnt__state_discounted__last_active_lteq_120_days      94
crdeal_deallife__loans_cnt__state_discounted__last_active_lteq_180_days      94
crdeal_deallife__loans_cnt__state_discounted__last_active_lteq_360_days      94
                                                                           ... 
crdeal_deallife__initial_amount_mean__state_clsd                           2215
crdeal_deallife__historical_max_overdue_debt__clsd_state                   2215
crdeal_deallife__initial_amount_sum__state_clsd                            2215
crdeal_deallife__current_max_dpd__donor_mfo                                3648
crdeal_deallife__current_max_dpd__donor_bnk                                4816
Length: 284, dtype: int64

In [8]:
data = data.fillna(data.mean())

In [9]:
missing = data.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)

In [10]:
missing

crdeal_deallife__first_loan_open_date                 94
crdeal_deallife__last_loan_open_date                  94
crdeal_deallife__first_loan_open_date__donor_mfo    1687
crdeal_deallife__last_loan_open_date__donor_mfo     1687
crdeal_deallife__first_loan_open_date__donor_bnk    1943
crdeal_deallife__last_loan_open_date__donor_bnk     1943
dtype: int64

In [11]:
data = data.drop(missing.index, axis=1)

# Splitting the data into train and test

In [12]:
test_data = data[data['target_flag'] == 'not determined']
test_data = test_data.drop('target_flag', axis=1)

test_data.head()

,crdeal_deallife__loans_cnt__dlrolesub_zaemshik__curr_uah,crdeal_deallife__loans_cnt__dlrolesub_zaemshik__curr_not_uah,crdeal_deallife__loans_cnt__dldonor_bnk,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_7_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_30_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_45_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_60_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_90_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_120_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_180_days,...,credres__credit_req_cnt__org_bnk__last_3_5_days,credres__credit_req_cnt__org_bnk__last_7_days,credres__credit_req_cnt__org_bnk__last_10_days,credres__credit_req_cnt__org_bnk__last_13_days,credres__credit_req_cnt__org_bnk__last_20_days,credres__credit_req_cnt__org_bnk__last_30_days,credres__credit_req_cnt__org_bnk__last_60_days,credres__credit_req_cnt__org_bnk__more_than_90_days,credres__credit_req_cnt__org_bnk__last_360_days,credres__credit_req_cnt__org_bnk__more_than_360_days
7,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
9,25.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,4.0,11.0,9.0,7.0
14,21.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,2.0,2.0,2.0,6.0,7.0,1.0
17,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
21,9.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0


In [13]:
train_data = data[data['target_flag'] != 'not determined']
train_data.head()

,target_flag,crdeal_deallife__loans_cnt__dlrolesub_zaemshik__curr_uah,crdeal_deallife__loans_cnt__dlrolesub_zaemshik__curr_not_uah,crdeal_deallife__loans_cnt__dldonor_bnk,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_7_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_30_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_45_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_60_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_90_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_120_days,...,credres__credit_req_cnt__org_bnk__last_3_5_days,credres__credit_req_cnt__org_bnk__last_7_days,credres__credit_req_cnt__org_bnk__last_10_days,credres__credit_req_cnt__org_bnk__last_13_days,credres__credit_req_cnt__org_bnk__last_20_days,credres__credit_req_cnt__org_bnk__last_30_days,credres__credit_req_cnt__org_bnk__last_60_days,credres__credit_req_cnt__org_bnk__more_than_90_days,credres__credit_req_cnt__org_bnk__last_360_days,credres__credit_req_cnt__org_bnk__more_than_360_days
0,good,40.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,5.0,4.0
1,good,72.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,14.0,6.0,11.0
2,good,77.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,5.0,9.0
3,good,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0
4,good,8.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


# Encoding target into 0's and 1's (0 - bad, 1 - good)

In [14]:
le = LabelEncoder()
le.fit(train_data['target_flag'])

LabelEncoder()

In [15]:
le.classes_

array(['bad', 'good'], dtype=object)

In [16]:
train_data['target_flag'] = le.transform(train_data['target_flag'])

# Selecting highly corellated features

In [17]:
cor = train_data.corr()

In [18]:
cor

,target_flag,crdeal_deallife__loans_cnt__dlrolesub_zaemshik__curr_uah,crdeal_deallife__loans_cnt__dlrolesub_zaemshik__curr_not_uah,crdeal_deallife__loans_cnt__dldonor_bnk,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_7_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_30_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_45_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_60_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_90_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_120_days,...,credres__credit_req_cnt__org_bnk__last_3_5_days,credres__credit_req_cnt__org_bnk__last_7_days,credres__credit_req_cnt__org_bnk__last_10_days,credres__credit_req_cnt__org_bnk__last_13_days,credres__credit_req_cnt__org_bnk__last_20_days,credres__credit_req_cnt__org_bnk__last_30_days,credres__credit_req_cnt__org_bnk__last_60_days,credres__credit_req_cnt__org_bnk__more_than_90_days,credres__credit_req_cnt__org_bnk__last_360_days,credres__credit_req_cnt__org_bnk__more_than_360_days
target_flag,1.000000,0.091743,0.013211,0.038861,-0.007049,-0.011811,-0.017213,-0.022600,-0.017219,-0.013722,...,-0.020655,-0.027994,-0.029913,-0.033918,-0.040779,-0.044781,-0.047482,0.061724,0.012885,0.063783
crdeal_deallife__loans_cnt__dlrolesub_zaemshik__curr_uah,0.091743,1.000000,0.022652,0.273456,-0.005998,0.014167,0.020049,0.023663,0.043557,0.051764,...,0.002438,0.008426,0.015355,0.018750,0.026693,0.042059,0.082420,0.379365,0.295388,0.322104
crdeal_deallife__loans_cnt__dlrolesub_zaemshik__curr_not_uah,0.013211,0.022652,1.000000,0.068526,0.001052,-0.000032,0.007995,0.007843,0.003092,0.002735,...,0.007526,0.007545,0.015456,0.016200,0.021692,0.019370,0.021928,0.040042,0.030510,0.039556
crdeal_deallife__loans_cnt__dldonor_bnk,0.038861,0.273456,0.068526,1.000000,0.029250,0.205050,0.277884,0.323573,0.403166,0.460923,...,0.044780,0.057475,0.071271,0.081803,0.108255,0.135666,0.194585,0.475663,0.378076,0.444309
crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_7_days,-0.007049,-0.005998,0.001052,0.029250,1.000000,0.323123,0.236427,0.189661,0.138114,0.117182,...,0.058297,0.081329,0.073869,0.072200,0.062868,0.054911,0.043022,0.010901,0.019944,0.011899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
credres__credit_req_cnt__org_bnk__last_30_days,-0.044781,0.042059,0.019370,0.135666,0.054911,0.212988,0.206939,0.196184,0.184562,0.180395,...,0.434945,0.571008,0.661063,0.732974,0.873283,1.000000,0.813793,0.162939,0.468698,0.107198
credres__credit_req_cnt__org_bnk__last_60_days,-0.047482,0.082420,0.021928,0.194585,0.043022,0.173926,0.244423,0.284395,0.264335,0.258730,...,0.343214,0.449135,0.523404,0.581142,0.696603,0.813793,1.000000,0.238205,0.616566,0.156267
credres__credit_req_cnt__org_bnk__more_than_90_days,0.061724,0.379365,0.040042,0.475663,0.010901,0.052234,0.074846,0.086743,0.117520,0.164420,...,0.055509,0.076927,0.093639,0.106141,0.126173,0.162939,0.238205,1.000000,0.768297,0.856923
credres__credit_req_cnt__org_bnk__last_360_days,0.012885,0.295388,0.030510,0.378076,0.019944,0.105179,0.149380,0.180100,0.232132,0.276797,...,0.179013,0.237775,0.281889,0.315962,0.385610,0.468698,0.616566,0.768297,1.000000,0.423432


In [19]:
cor_target = abs(cor["target_flag"])

In [20]:
relevant_features = cor_target[cor_target>0.5]
relevant_features

target_flag    1.0
Name: target_flag, dtype: float64

No features highly corellated with the target found

# Splitting train data into train and validation datasets

In [21]:
y = train_data['target_flag']
X = train_data.drop('target_flag', axis=1)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [23]:
del X, y, le, train_data, data, relevant_features, cor, cor_target
gc.collect()

100

# Scaling data

In [24]:
scaler = StandardScaler().fit(X_train)

In [25]:
X_train = scaler.transform(X_train)

# Fitting a Logistic Regression model

In [26]:
log_reg = LogisticRegression()

** Using RandomizedSearchCV **

In [27]:
penalty = ['none']
tol = [1e-4, 1e-3, 1e-2, 1e-1]
C = [float(x) for x in np.linspace(0, 1, num = 10)]
class_weight = ['balanced'] 
solver = ['sag']
max_iter = [int(x) for x in np.linspace(500, 1000, num = 10)]


random_grid = {
    'penalty' : penalty,
    'tol' : tol,
    'max_iter' : max_iter,
    'C': C,
    'solver' : solver,
    'class_weight' : class_weight
}


pprint(random_grid)

{'C': [0.0,
       0.1111111111111111,
       0.2222222222222222,
       0.3333333333333333,
       0.4444444444444444,
       0.5555555555555556,
       0.6666666666666666,
       0.7777777777777777,
       0.8888888888888888,
       1.0],
 'class_weight': ['balanced'],
 'max_iter': [500, 555, 611, 666, 722, 777, 833, 888, 944, 1000],
 'penalty': ['none'],
 'solver': ['sag'],
 'tol': [0.0001, 0.001, 0.01, 0.1]}


In [28]:
clf = RandomizedSearchCV(estimator = log_reg, param_distributions = random_grid, n_iter = 5, cv = 3, verbose = 2, random_state = 42, n_jobs = -1, scoring = 'roc_auc')

In [29]:
%%time
clf.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:  1.6min remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.6min finished
CPU times: user 49.2 s, sys: 363 ms, total: 49.5 s
Wall time: 3min 24s


RandomizedSearchCV(cv=3, estimator=LogisticRegression(), n_iter=5, n_jobs=-1,
                   param_distributions={'C': [0.0, 0.1111111111111111,
                                              0.2222222222222222,
                                              0.3333333333333333,
                                              0.4444444444444444,
                                              0.5555555555555556,
                                              0.6666666666666666,
                                              0.7777777777777777,
                                              0.8888888888888888, 1.0],
                                        'class_weight': ['balanced'],
                                        'max_iter': [500, 555, 611, 666, 722,
                                                     777, 833, 888, 944, 1000],
                                        'penalty': ['none'], 'solver': ['sag'],
                                        'tol': [0.0001, 0.001, 0.01, 0.1]},

In [30]:
print('Best Score: ', clf.best_score_) 
print('Best Params: ', clf.best_params_)

Best Score:  0.6471508511629334
Best Params:  {'tol': 0.0001, 'solver': 'sag', 'penalty': 'none', 'max_iter': 500, 'class_weight': 'balanced', 'C': 0.7777777777777777}


In [31]:
pred = clf.predict(X_test)

In [32]:
del X_train, y_train
gc.collect()

381

# Measuring performance

In [46]:
fpr, tpr, thresholds = roc_curve(y_test, pred)
print(auc(fpr, tpr))

NameError: name 'y_test' is not defined

In [34]:
accuracy_score(y_test, pred)

0.6616843920666858

In [35]:
f1_score(y_test, pred)

0.7816191477518708

In [36]:
del X_test, y_test, pred
gc.collect()

2864

# Making predictions on test data

In [37]:
target_flag = clf.predict(test_data)

In [38]:
target_flag

array([1, 1, 1, ..., 1, 1, 1])

In [39]:
test_data.head()

,crdeal_deallife__loans_cnt__dlrolesub_zaemshik__curr_uah,crdeal_deallife__loans_cnt__dlrolesub_zaemshik__curr_not_uah,crdeal_deallife__loans_cnt__dldonor_bnk,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_7_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_30_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_45_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_60_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_90_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_120_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_180_days,...,credres__credit_req_cnt__org_bnk__last_3_5_days,credres__credit_req_cnt__org_bnk__last_7_days,credres__credit_req_cnt__org_bnk__last_10_days,credres__credit_req_cnt__org_bnk__last_13_days,credres__credit_req_cnt__org_bnk__last_20_days,credres__credit_req_cnt__org_bnk__last_30_days,credres__credit_req_cnt__org_bnk__last_60_days,credres__credit_req_cnt__org_bnk__more_than_90_days,credres__credit_req_cnt__org_bnk__last_360_days,credres__credit_req_cnt__org_bnk__more_than_360_days
7,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
9,25.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,4.0,11.0,9.0,7.0
14,21.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,2.0,2.0,2.0,6.0,7.0,1.0
17,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
21,9.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0


In [40]:
data = pd.concat([pd.Series(target_flag, name='target_flag', index=test_data.index), test_data], axis=1)

In [41]:
data.head()

,target_flag,crdeal_deallife__loans_cnt__dlrolesub_zaemshik__curr_uah,crdeal_deallife__loans_cnt__dlrolesub_zaemshik__curr_not_uah,crdeal_deallife__loans_cnt__dldonor_bnk,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_7_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_30_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_45_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_60_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_90_days,crdeal_deallife__loans_cnt__dldonor_bnk__opened_lteq_120_days,...,credres__credit_req_cnt__org_bnk__last_3_5_days,credres__credit_req_cnt__org_bnk__last_7_days,credres__credit_req_cnt__org_bnk__last_10_days,credres__credit_req_cnt__org_bnk__last_13_days,credres__credit_req_cnt__org_bnk__last_20_days,credres__credit_req_cnt__org_bnk__last_30_days,credres__credit_req_cnt__org_bnk__last_60_days,credres__credit_req_cnt__org_bnk__more_than_90_days,credres__credit_req_cnt__org_bnk__last_360_days,credres__credit_req_cnt__org_bnk__more_than_360_days
7,1,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
9,1,25.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,4.0,11.0,9.0,7.0
14,1,21.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,2.0,2.0,2.0,6.0,7.0,1.0
17,0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
21,1,9.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0


In [42]:
data.to_excel('prediction.xlsx', index=False)